# **Работаем над классификацией резюме.**
### На основе датасетов вида: "Category", "Resume" мы создаем датасет вида: "Category", "Resume", "Label", т.е. добавляем еще один столбец, в котором указываем принадлежность каждого резюме к одному из 7-ми классов (см. ниже)

In [ ]:
import os
os.mkdir("data")

In [ ]:
# Скачиваем архив с резюме с гугл-диска
import gdown
url = 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=17VltEkh-3TE4DoW8bxCaPLzvsz0lMRPP'
gdown.download(url, '/content/')

Downloading...
From: https://drive.google.com/uc?export=download&confirm=no_antivirus&id=17VltEkh-3TE4DoW8bxCaPLzvsz0lMRPP
To: /content/resumes_corpus.zip
100%|██████████| 89.4M/89.4M [00:01<00:00, 49.4MB/s]


'/content/resumes_corpus.zip'

In [ ]:
! unzip -q /content/resumes_corpus.zip  -d data

In [ ]:
# В скаченном архиме есть проблема: одна и та же специальность (например Web Developer) может фигурировать под десятком похожих названий,
# например: 'Web App Developer', 'Web Applications', 'WEB DEVELOPER' и т.п, поэтому нужна некая нормализация специальностей:
def labels_normalization(orig_label):
  if ('Project Manager' in orig_label) or \
      ('IT Manager' in orig_label) or \
      ('PROJECT MANAGER' in orig_label):
      return 'Project Manager'

  elif ('Web Developer' in orig_label) or \
        ('Web App Developer' in orig_label) or \
        ('Front End' in orig_label) or \
        ('Web Applications' in orig_label) or \
        ('WEB DEVELOPER' in orig_label) or \
        ('Front- End Developer' in orig_label) or \
        ('Web developer' in orig_label) or \
        ('WEB DEVELOPER' in orig_label) or \
        ('JavaScript' in orig_label):
        return 'Web Designing'

  elif ('Back End Developer' in orig_label):
      return 'BackEnd Developer'

  elif ('UI designer' in orig_label) or \
        ('UI/UX' in orig_label):
      return 'UI/UX designer'

  elif ('Full Stack' in orig_label):
      return 'Full Stack Developer'

  elif ('Java Developer' in orig_label) or \
        ('Java software engineer' in orig_label) or \
        ('JAVA DEVELOPER' in orig_label):
      return 'Java Developer'

  elif ('Python Developer' in orig_label):
      return 'Python Developer'

  elif ('Data Scientist' in orig_label) or \
       ('Data Science' in orig_label) or \
        ('Machine Learning' in orig_label):
      return 'Data Science'

  elif ('Database Administrator' in orig_label) or \
        ('Database Analyst' in orig_label):
      return 'Database Administrator'

  elif ('System Administrator' in orig_label) or \
        ('Systems and Network Administrator' in orig_label) or \
        ('Systems Engineer' in orig_label) or \
        ('Systems Administrator' in orig_label) or \
        ('SYSTEMS ADMINISTRATOR' in orig_label):
      return 'System Administrator'

  elif ('Systems Analyst' in orig_label):
      return 'Systems Analyst'

  elif ('Software Engineer' in orig_label) or \
        ('Software developer' in orig_label) or \
        ('Software Developer' in orig_label) or \
        ('Software Development Engineer' in orig_label):
      return 'Software Developer'

  elif ('Network Administrator' in orig_label) or \
        ('Network Communications Administrator' in orig_label) or \
        ('Network Engineer' in orig_label) or \
        ('NETWORK ADMINISTRATOR' in orig_label):
      return 'Network Administrator'

  elif ('IT Support Engineer' in orig_label):
      return 'IT Support Engineer'

  elif ('.NET Developer' in orig_label):
      return 'DotNet Developer'

  elif ('IT Service Manager' in orig_label):
      return 'IT Service Manager'

  elif ('IT Security Engineer' in orig_label) or \
        ('IT Security Analyst' in orig_label) or \
        ('Security Manager' in orig_label) or \
        ('Network Security Engineer' in orig_label) or \
        ('Network Security Administrator' in orig_label) or \
        ('Security Analyst' in orig_label) or \
        ('IT SECURITY' in orig_label) or \
        ('Cyber Security' in orig_label):
        return 'Network Security Engineer'

  else:
      return ''

In [ ]:
# Всего в архиве оказалось несколько десятков тысяч резюме - у нас нет финансовых возможностей работать с таким огромным датасетом,
# поэтому мы сокращаем количество резюме с помощью мини-нормализации:

def labels_normalization_mini(orig_label):
  if ('WEB DEVELOPER' in orig_label):
      return 'Web Designing'

  elif ('Back End Developer' in orig_label):
      return 'BackEnd Developer'

  elif ('UI designer' in orig_label) or \
        ('UI/UX' in orig_label):
      return 'UI/UX designer'

  elif ('Full Stack' in orig_label):
      return 'Full Stack Developer'

  elif ('Data Scientist' in orig_label) or \
       ('Data Science' in orig_label) or \
        ('Machine Learning' in orig_label):
      return 'Data Science'

  elif ('Database Analyst' in orig_label):
      return 'Database Analyst'

  elif ('Systems Analyst' in orig_label):
      return 'Systems Analyst'

  elif ('SSoftware Engineer' in orig_label):
      return 'Software Developer'

  elif ('IT Support Engineer' in orig_label):
      return 'IT Support Engineer'

  elif ('.NET Developer' in orig_label):
      return 'DotNet Developer'

  elif ('IT Service Manager' in orig_label):
      return 'IT Service Manager'

  elif ('IT Security Engineer' in orig_label) or \
        ('Network Security Engineer' in orig_label):
        return 'Network Security Engineer'

  else:
      return ''

In [ ]:
# Кроме датасета выше мы используем еще один датасет поменьше:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauravduttakiit/resume-dataset")

100%|██████████| 383k/383k [00:00<00:00, 49.4MB/s]

Extracting files...


In [ ]:
import pandas as pd

df=pd.read_csv(path + '/UpdatedResumeDataSet.csv')

In [ ]:
df['Category'].unique()

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [ ]:
# В этом втором датасете много специальностей, которые нам не нужны: 'HR', 'Advocate', 'Arts' и т.п.
# Поэтому мы избавляемся от них:
df.drop(df[(df['Category'] == 'HR') | (df['Category'] == 'Operations Manager') | (df['Category'] == 'Hadoop') | (df['Category'] == 'ETL Developer') | (df['Category'] == 'Database') | (df['Category'] == 'Advocate') | (df['Category'] == 'Arts') | (df['Category'] == 'Sales') | (df['Category'] == 'Health and fitness') | (df['Category'] == 'Civil Engineer') | (df['Category'] == 'Electrical Engineering') | (df['Category'] == 'PMO') | (df['Category'] == 'Blockchain') | (df['Category'] == 'Mechanical Engineer')].index, inplace=True)
df['Category'].unique()

array(['Data Science', 'Web Designing', 'Java Developer',
       'Business Analyst', 'SAP Developer', 'Automation Testing',
       'Python Developer', 'DevOps Engineer', 'Network Security Engineer',
       'DotNet Developer', 'Testing'], dtype=object)

In [ ]:
df.Category.value_counts()

,count
Category,
Java Developer,84
Testing,70
DevOps Engineer,55
Python Developer,48
Web Designing,45
Data Science,40
Business Analyst,28
DotNet Developer,28
Automation Testing,26


In [ ]:
df.shape

(473, 2)

In [ ]:
# В первом датасете название специальности содержится непосредственно в тексте резюме - нужно извлечь его оттуда.
# Обычно название содержится в первой строке и заключено в <  >, пишем функцию, которая извлекает название специальности из первой строки из <  >
# Название специальности и сам текст резюме мы добавляем ко второму датасету (датафрейму df)
for resume in os.listdir('/content/data'):
    if resume.endswith('txt'):
        with open('/content/data/' + resume, 'r', encoding='windows-1252') as f:
            lines = f.readlines()
            f.close()
        with open('/content/data/' + resume, 'r', encoding='windows-1252') as f:
            resume = f.read()
            f.close()
        values = lines[0].split(' <')
        category = labels_normalization_mini(values[0])
        if category == '':
            continue
        df = df._append({'Category': category, 'Resume': resume}, ignore_index=True)

In [ ]:
df.Category.value_counts()

,count
Category,
Full Stack Developer,947
Data Science,344
Systems Analyst,341
UI/UX designer,266
Web Designing,215
Network Security Engineer,131
Java Developer,84
DotNet Developer,78
Database Analyst,75


In [ ]:
df.shape

(2772, 2)

In [ ]:
# Очищаем тексты с резюме:
import re
import string
def clear_fun(text):
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
   # text = re.sub('\s+', '', text)
    return text

df['Resume'] = df['Resume'].apply(clear_fun)

In [ ]:
df['Category'][1001]

'Systems Analyst'

In [ ]:
# Выводим текст произвольного резюме:
df['Resume'][1001]

'senior graphic designer   front end developer senior graphic designer    span class  hl  front  span   span class  hl  end  span   span class  hl  developer  span  miami  fl extremely creative  and multi talented designer with    years of experience in graphic design  printing  web design  motion graphics   video being an artist for    years has lead me to get into custom hand lettering  character design   branding  i m an artist at heart  creating is my passion  authorized to work in the us for any employer work experience senior graphic designer   front end developer pop creative   miami  fl february   to present   managed design team of   people    oversaw all final designs before sending to ensure quality    handled quality assurance for most sites    programmed front end of all sites in company pipeline     oversaw company s printing department    managed server    stay up ro date with industry developments and tools senior graphic designer   store manager korum customs   miami b

In [ ]:
# Для нашего проекта нам нужно классифицировать все резюме по неким классам, отражающим определенные недоработки (слабые места) каждого резюме.
# Для этого мы будем использовать ChatGPT и OpenAI API
!pip install langchain-community
!pip install faiss-cpu
!pip install tiktoken

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Создаем пустой датафрейм, но уже со столбцом меток класса:
df_with_labels = pd.DataFrame(columns=['Category', 'Resume', 'Label'])

In [ ]:
# Все резюме будем классифицировать по семи классам:
def cv_7labels_query(query_with_chunks, cv_category):

    query = f''' Я сейчас представлю тебе резюме специалиста, претендующего на позицию:

                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                  {cv_category}
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                  Мне нужно, чтобы ты максимально подробно проанализировал это резюме, а также отнес его к одному из семи классов.
                  Класс 0: резюме составлено правильно и корректно с точки зрения работодателя, особых замечаний нет.
                  Класс 1: недостаточно полно указаны технические навыки, опыт и достижения
                  Класс 2: нет информации о предыдущих местах работы
                  Класс 3: недостаточно подробно перечислены основные технологии и инструменты, с которыми умеет работать автор резюме
                  Класс 4: недостаточно информации про образование, в том числе и про дополнительное (онлайн курсы, повышение квалификации, полученные сертификаты и так далее)
                  Класс 5: резюме недостаточно краткое и лаконичное
                  Класс 6: нет информации о готовности к постоянному обучению и развитию (саморазвитию), о готовности работать в команде, об аналитических навыках и так далее
                  Не очень строго оценивай резюме по классу 1.
                  И вообще будь помягче - сейчас на рынке большой дефицит специалистов и работодатель на некоторые моменты может "закрыть глаза".
                  Если тебе кажется, что резюме не относится к классу 0 - подумай еще раз.
                  После всестороннего анализа резюме верни мне только одну цифру: номер класса, к которому ты отнес это резюме - от 0 до 6.
                  Запомни: возвращай только одну цифру и больше ничего! Это очень важно!
                  Вот это резюме:
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                  {query_with_chunks}
                '''
    return query

In [ ]:
# Для работы с объемными резюме нам понадобится text_splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700,
                                               chunk_overlap=200,
                                               length_function=len)

In [ ]:
# Основная функция-классификатор
def openai(openai_api_key, chunks, analyze):

    # Использование сервиса OpenAI для эмбеддингов
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

    # Библиотека Facebook AI Similarity Serach помогает нам преобразовывать текстовые данные в числовой вектор
    vectorstores = FAISS.from_texts(chunks, embedding=embeddings)

    # Сравниваем запрос и фрагменты, это позволяет выбрать «К» наиболее похожих фрагментов на основе их оценок схожести.
    docs = vectorstores.similarity_search(query=analyze, k=3)

    # Создаем OpenAI объект
    llm = ChatOpenAI(model='chatgpt-4o-latest', api_key=openai_api_key)
    #llm = ChatOpenAI(model='gpt-4o', api_key=openai_api_key)

    # Конвейер вопросов и ответов (QA), использующий функцию load_qa_chain
    chain = load_qa_chain(llm=llm, chain_type='stuff')

    response = chain.run(input_documents=docs, question=analyze)
    return response

In [ ]:
# Процесс классификации
count = 0
for row in df.itertuples():
  category = row.Category
  resume = row.Resume
  # Разбиваем длинные тексты резюме на небольшие фрагменты.
  chunks = text_splitter.split_text(text=row.Resume)
  # Проводим нормализацию специальностей
  query = cv_7labels_query(query_with_chunks=chunks, cv_category=row.Category)
  # Получаем с помощью openai chatgpt метку класса
  label = openai(openai_api_key=openai_api_key, chunks=chunks, analyze=query)
  # Добавляем в исходный пустой df_with_labels построчно данные Category, Resumе и Label
  df_with_labels = df_with_labels._append({'Category': category, 'Resume': resume, 'Label': int(label)}, ignore_index=True)
  print(str(count) + ': ' + category + ' = ' + label)
  count = count + 1

In [ ]:
# Сохраняем полученный датасет в файл
df_with_labels.to_csv('df_with_labels.csv')